## Dataset
https://www.kaggle.com/datasets/msambare/fer2013

### 1.- Librerias necesarias

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

### 2.- Cargar las imagenes del dataset

In [ ]:
# Ruta al directorio que contiene las imágenes
directorio_datos = "/Users/cesar/Documents/TEC DE CULIACAN/8 SEMESTRE TEC/IA/Modulo 4/TAREA 1/dataset"

# Cargamos las imágenes y etiquetas de forma automática
datos_brutos = tf.keras.utils.image_dataset_from_directory(
    directory=directorio_datos,
    labels='inferred',        
    label_mode='int',         # Etiquetas como enteros
    image_size=(256, 256),    
    batch_size=32,            
    shuffle=True              
)

Found 1148 files belonging to 6 classes.


### 3. Preprocesamiento y aumento de datos

In [ ]:
def aplicar_aumentos(imagen, etiqueta):
    imagen = tf.image.convert_image_dtype(imagen, tf.float32)

    # Aplicar aumento de brillo y contraste de forma aleatoria
    if tf.random.uniform(()) > 0.5:
        imagen = tf.image.random_brightness(imagen, max_delta=0.05)
    if tf.random.uniform(()) > 0.5:
        imagen = tf.image.random_contrast(imagen, lower=0.95, upper=1.05)

    # Rotación aleatoria de la imagen
    rotaciones = tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32)
    imagen = tf.image.rot90(imagen, k=rotaciones)

    # Simulación de zoom mediante padding y recorte aleatorio
    imagen = tf.image.resize_with_crop_or_pad(imagen, 270, 270)     # Añade borde
    imagen = tf.image.random_crop(imagen, size=[256, 256, 3])       # Recorta de nuevo

    return imagen, etiqueta

AUTOTUNE = tf.data.AUTOTUNE
# Aplicamos las transformaciones al dataset
datos_preprocesados = (
    datos_brutos
    .unbatch()  
    .map(aplicar_aumentos, num_parallel_calls=AUTOTUNE)  
    .batch(32)  
    .prefetch(AUTOTUNE) 
)